In [1]:
import json

from kafka import KafkaConsumer

### Configuration Parameters 

> **TODO:** Change the configuration prameters to the appropriate values for your setup.

In [2]:
config = dict(
    bootstrap_servers=['kafka.kafka.svc.cluster.local:9092'],
    first_name='Arindam',
    last_name='Samanta'
)

config['client_id'] = '{}{}'.format(
    config['last_name'], 
    config['first_name']
)
config['topic_prefix'] = '{}{}'.format(
    config['last_name'], 
    config['first_name']
)

config

{'bootstrap_servers': ['kafka.kafka.svc.cluster.local:9092'],
 'first_name': 'Arindam',
 'last_name': 'Samanta',
 'client_id': 'SamantaArindam',
 'topic_prefix': 'SamantaArindam'}

Create a consumer without subscribing to any particular topic

In [3]:
general_consumer = KafkaConsumer(
    bootstrap_servers=config['bootstrap_servers']
)

List all topics you are currently allowed to view

In [11]:
general_consumer.topics()

{'DoeJohn-accelerations',
 'DoeJohn-locations',
 'DoeJohn-simple',
 'GonzalezKimberly-accelerations',
 'GonzalezKimberly-joined',
 'GonzalezKimberly-locations',
 'GonzalezKimberly-simple',
 'GonzalezKimberly-windowed',
 'HoffmannLaura-accelerations',
 'HoffmannLaura-locations',
 'KalaikkovanVasanthakumar-accelerations',
 'KalaikkovanVasanthakumar-locations',
 'MohammedShabuddinRamizuddin-accelerations',
 'MohammedShabuddinRamizuddin-locations',
 'MuleyTushar-accelerations',
 'MuleyTushar-accelrations',
 'MuleyTushar-locations',
 'MuthusamyGaneshkumar-accelerations',
 'MuthusamyGaneshkumar-locations',
 'NagarajVinay-accelerations',
 'NagarajVinay-locations',
 'PickCameron-accelerations',
 'PickCameron-locations',
 'SamantaArindam-locations',
 'SelvarajDavid-accelerations',
 'SelvarajDavid-locations',
 'SumbarajuAditya-accelerations',
 'SumbarajuAditya-locations'}

Close the consumer, waiting indefinitely for any needed cleanup.

In [5]:
general_consumer.close()

In [6]:
def create_kafka_consumer(topics, config=config):
    bootstrap_servers = config['bootstrap_servers']
    client_id = config['client_id']
    topic_prefix = config['topic_prefix']    
    topic_list = ['{}-{}'.format(topic_prefix, topic) for topic in topics]
    
    return KafkaConsumer(
        *topic_list,
        client_id=client_id,
        bootstrap_servers=bootstrap_servers,
        auto_offset_reset='earliest',
        enable_auto_commit=False,
        value_deserializer=lambda x: json.loads(x)
    )

consumer = create_kafka_consumer(['locations', 'accelerations'])

Gets a list of this consumer's current subscriptions

In [7]:
consumer.subscription()

{'SamantaArindam-accelerations', 'SamantaArindam-locations'}

The following function prints messages from the current consumer subscriptions. It will continue until manually stopped. 

In [8]:
def print_messages(consumer=consumer):
    try:
        for message in consumer:
                msg_metadata = 'Message metadata: {}:{}:{}'.format(
                    message.topic, message.partition, message.offset
                )

                if message.key is not None:
                    msg_key = message.key.decode('utf-8')
                else:
                    msg_key = ''
                msg_value = json.dumps(message.value, indent=2)
                msg_value = '\n'.join(['  {}'.format(value) for value in msg_value.split('\n')])

                print('Message metadata:')
                print('  Topic: {}'.format(message.topic))
                print('  Partition: {}'.format(message.partition))
                print('  Offset: {}'.format(message.offset))
                print('Message Key: {}'.format(msg_key))
                print('Message Value:')
                print(msg_value)
                print()
    except KeyboardInterrupt:
        print("STOPPING MESSAGE CONSUMER")
        
print_messages()

Message metadata:
  Topic: SamantaArindam-locations
  Partition: 0
  Offset: 0
Message Key: e79e12e9c70746389432c1f774a3414f
Message Value:
  {
    "key1": "value1",
    "key2": "value2"
  }

STOPPING MESSAGE CONSUMER


Close the consumer, waiting indefinitely for any needed cleanup.

In [10]:
consumer.close()